In [6]:
import pandas as pd
import random
from openai import OpenAI

df = pd.read_csv(r"D:\LLMTables\LLMTablesQA\Question Generation\TestTables_5\sportset_coldtemp_30_13.csv")
client = OpenAI()
row = df.sample(1, random_state=random.randint(1, 1000), ignore_index=True)
def generate_qa_pairs(row, num_pairs=1):
    
    # Temporal
    prompt = f"""
    Objective:
    Create complex, logic-driven questions based on a dataset row that represents a real-world scenario. Each question should require multi-step reasoning, utilizing both structured and unstructured elements of the row. The questions should not directly reveal answers, instead encouraging logical deductions and practical reasoning.
    Row Data: {row.to_dict(orient='records')[0]}
    Guidelines for Question Generation:
    1. Inter-Cell Logic: Questions must connect multiple data points within the row, encouraging indirect reasoning between structured and unstructured components.
    2. Structured + Unstructured: Utilize both types of data (e.g., scores, descriptions) when formulating questions.
    3. Avoid Direct Retrieval: Do not allow answers to be directly extracted from the row; ensure questions require reasoning.
    4. Multi-hop Reasoning: Answers should require multiple logical operations or connections to arrive at.
    5. Unique, Deterministic Answer: Ensure only one valid answer can be derived from the data, based on logical connections.
    6. Simplified Language: Keep the language of questions concise and clear, avoiding unnecessary complexity.
    7. No Word Overlap: Avoid using the same words or phrases from the row in the questions to maintain a level of difficulty.
    Single Subtle Hint: Include one subtle clue to guide the reasoning, but ensure further deductions are needed for the correct answer.
    9. Semantic Diversity: Aim for semantic diversity in the questions, avoiding repetition of themes or concepts.
    
    Example Questions + why they are good questions:
    1. In which month did the NY Knicks team play their games, considering it was during the final quarter of the year?
    Reason: There are just the right amount of giveaways to answer the question. You can see that the team is New York knocks and find their games at the end of the year.
    2. Where outside USA has the boston team played?
    Reason: "Outside USA" is a good marker for indirect clues which can confuse the person answering the question.
    3. In which city did a team achieve a victory with a margin of 17 points on the last Tuesday of October? 
    Reaso8. n: There are two markers point margin and day which can help the model to filter out the information without giving too much information away.
    4. Where was a game held that featured a standout performance of 41 points and nearly full attendance?

    Take inspiration from these questions and try to make the question but not so vague that multiple answers exist in the table for that question.
    
    Generate {num_pairs} such questions. Ensure that the answer is given in maximum 3-5 words.

    Answer format:
    Q: <Generated Question>
    A: <Generated Answer>
    """

    # Define your messages
    messages = [
        {"role": "system", "content": "You are an expert in generating complex questions from tabular data. Your task is to create questions that require analyzing inter-cell relationships within the provided table and can be answered directly or indirectly using the table's content. The questions should leverage both structured and unstructured components of the table data."},
        {"role": "user", "content": prompt}
    ]

    # Make the API call
    completion = client.chat.completions.create(
        model="gpt-4o-mini",  # or the model you are using
        messages=messages,
        temperature=0.5
    )

    # Print the response
    print(completion.choices[0].message.content)

print(row)

generate_qa_pairs(row, 10)

   day     month  year dayname  season          stadium      city     state  \
0   10  February  2017  Friday    2016  Barclays Center  Brooklyn  New York   

   attendance  capacity  game_id  \
0       15400     17700     3145   

                                             summary  
0  The Miami Heat defeated the Brooklyn Nets , 10...  
Q: Which location hosted a matchup where one team was seeking to avoid a dozen consecutive losses?
A: Barclays Center

Q: During what season did a game occur where the winning team had a record of 24 wins and 30 losses?
A: 2016

Q: In which city did a team with a home advantage struggle to maintain possession, resulting in numerous turnovers?
A: Brooklyn

Q: What was the attendance at the venue where the home team faced a significant scoring deficit in the second half?
A: 15,400

Q: Which team was looking for its 13th consecutive victory while competing against a struggling opponent?
A: Miami Heat

Q: What was the point difference in the final score 

In [7]:
import pandas as pd
df = pd.read_csv("D:\LLMTables\LLMTablesQA\Question Generation\TestTables_5\sportset_coldtemp_30_13.csv")

string = ''
string = '/*\n'
col_list = df.columns.values.tolist()
string += 'col : ' + ' | '.join(df.columns) + '\n'
for row_id, row in df.iterrows():
    string += f'row {row_id} : '
    for column_id, header in enumerate(df.columns):
        string += str(row[header])
        if column_id != len(df.columns) - 1:
            string += ' | '
    string += '\n'
string += '*/\n'

In [10]:
# Prompt taken from End-to-End Qa in Chain of Table paper
answer_prompt = f"""
Here is the table to answer this question. Answer the question as an entity. Return an explanation on how you reached the answer.
{string}
Question: In which city did a team with a home advantage struggle to maintain possession, resulting in numerous turnovers?
The answer is: 
"""

# Define your messages
messages = [
    {"role": "system", "content": "You are an expert in answering questions from tabular data."},
    {"role": "user", "content": answer_prompt}
]

# Make the API call
completion = client.chat.completions.create(
    model="gpt-4o-mini",  # or the model you are using
    messages=messages,
    temperature = 0.0
)

# Print the response
print(completion.choices[0].message.content)

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-fRla5InIDQaDKnPLyFBX0VHH on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 7m12s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}